<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px"> 
# News Articles Clustering Using KMeans

## Overview


   **In this project we will try to cluster news articles into different categories automatically using unsupervised machine learning algorithm which is KMeans. The dataset consists of 4130 news articles in diffrent categories like business, entertainment, politics, sport, ect.**

<img src="https://miro.medium.com/max/700/1*YWEqFeKKKzDiNWy5UfrTsg.png" style="float: center; margin: 20px; height: 300px"> 

In [3]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [4]:
# read data
df = pd.read_csv('news_data.csv')
data = df['news_text']
data.head()

0    حذّرت الهيئة العامة للغذاء والدواء من منتج عسل...
1    أقامت حكومة مدينة يوكوهاما، والمعهد الوطني للد...
2    أظهر مقطع فيديو متداول آخر للفتاة المقيمة، وهي...
3    أنقذت شجاعة مواطن اليوم فريقًا لشركة متعاقدة م...
4    استقبل مدير الدفاع المدني بمنطقة الرياض، اللوا...
Name: news_text, dtype: object

### Data Cleaning Stopwords and Stemming
**This section is focused on defining functions to manipulate the synopses. load nltk.corpus's list of Arabic stop words. Stop words are words like {'آه', 'آها', 'آي', which don't convey significant meaning.**

In [5]:
# data cleaning

# create instance of ISRIStemmer() class to stem arabic words
st = ISRIStemmer()
# get the stop words list
arb_stopwords = set(stopwords.words("arabic"))

def stem_string(a_string):
    stemmed = []
    # يستخدم يلعب
    # خدم لعب
    try:
        for w in a_string.split(' '):
            stemmed.append(st.stem(w))
        return ' '.join(stemmed)
    except Exception as e:
        print(e)
        return ''

def remove_urls(a_string):  # 'Hi all https://twitter.com' =-> Hi All
    return re.sub(r"http\S+", "", a_string).strip()


def just_ar_words(a_string):
    return " ".join(re.findall(r'[\u0621-\u064A]+', a_string)).strip()


def clean(a_string):
    s = remove_urls(a_string)
    s = just_ar_words(s)
    s = stem_string(s)
    return s

### Arabic Stop Words


In [6]:
from nltk.corpus import stopwords
arb_stopwords = set(stopwords.words("arabic"))
print(arb_stopwords)

{'لك', 'إذ', 'بس', 'ذلكم', 'لكي', 'على', 'هاتين', 'هن', 'كيت', 'هذين', 'لستن', 'ليت', 'حيث', 'ها', 'أو', 'كأي', 'هل', 'شتان', 'فيما', 'لستما', 'كيفما', 'أين', 'إن', 'ألا', 'إليك', 'هلا', 'إذا', 'هؤلاء', 'لسن', 'غير', 'هذان', 'هاتي', 'اللتين', 'ريث', 'بعض', 'لهن', 'هنا', 'أنتما', 'لهم', 'وإذ', 'بيد', 'ثم', 'ذاك', 'عن', 'حيثما', 'هاته', 'بها', 'ذواتا', 'لكنما', 'كليهما', 'تلكما', 'اللائي', 'بهم', 'ذا', 'بماذا', 'فيها', 'لهما', 'فإن', 'اللتان', 'ذانك', 'ليس', 'هكذا', 'لعل', 'إي', 'آي', 'كل', 'بكم', 'ذو', 'لكم', 'إلى', 'لن', 'ذه', 'كلما', 'هيا', 'ولكن', 'فيم', 'هاهنا', 'مه', 'التي', 'إيه', 'فمن', 'اللذان', 'لنا', 'هذه', 'هاتان', 'أولئك', 'لدى', 'حين', 'كأين', 'بين', 'اللذين', 'متى', 'هذا', 'كيف', 'إليكما', 'أقل', 'ليست', 'أم', 'هناك', 'نعم', 'ما', 'سوف', 'عما', 'بل', 'لو', 'اللواتي', 'اللتيا', 'كليكما', 'أنتن', 'ته', 'من', 'ليستا', 'ذلك', 'مهما', 'ذين', 'عند', 'ممن', 'ذلكن', 'ذات', 'له', 'هم', 'وإن', 'بمن', 'كأنما', 'لسنا', 'وما', 'كي', 'اللاتي', 'عل', 'أما', 'تينك', 'بهما', 'كلا', 'لي', '

In [7]:
# apply cleaning data
# loop throw each row and apply clean function 
data_cleaned = data.apply(lambda x: clean(str(x)))
data_cleaned.head()

0    حذ رت هيئ عمة غذء دوء من نتج عسل علم جرة عسل ص...
1    قمت حكم دين وكو عهد وطن درس سان جمع اكت ياب ؤس...
2    ظهر قطع يدو داول اخر فتة قيم وهي حدث بصت عال و...
3    قذت شجع وطن اليوم فرق ا لشر عاقد مع حدى ادر حك...
4    قبل دير دفع دني نطق ريض لوء خلد حرق دير درة عم...
Name: news_text, dtype: object

In [8]:
data_cleaned[0]

'حذ رت هيئ عمة غذء دوء من نتج عسل علم جرة عسل صحي نظر لاحتوائه على ادة دئي خدم وصف طبة وقل هيئ عسل صحي الذي نجه شرك في الز خلف لئح فنة عمد عسل نحل رقم سبب حوئ على ادة دئي تادالافيل التي خدم وجب وصف طبة وأص هلك جنب نتج خلص مما لدي منه فور حفظ على صحت شير الى انه تبع نفذ جرء أكد من خلو سوق من نتج'

### CountVectorizer, TfidfTransformer, KMeans Modeling Using Pipeline

In [9]:
pipeline = Pipeline([('vect', CountVectorizer(stop_words=arb_stopwords)),
                 ('tfidf', TfidfTransformer()),
                 ('kmeans', KMeans(n_clusters=18))])
pipeline.fit(data_cleaned)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words={'آه', 'آها', 'آي', 'أف', 'أقل',
                                             'أكثر', 'ألا', 'أم', 'أما', 'أن',
                                             'أنا', 'أنت', 'أنتم', 'أنتما',
                                             'أن...
                                 strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2

In [10]:
labels = pipeline['kmeans'].labels_
set(labels)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}

In [11]:
df['labels'] = labels
new_data = df[['news_text', 'labels']]
new_data.head()

,news_text,labels
0,حذّرت الهيئة العامة للغذاء والدواء من منتج عسل...,0
1,أقامت حكومة مدينة يوكوهاما، والمعهد الوطني للد...,6
2,أظهر مقطع فيديو متداول آخر للفتاة المقيمة، وهي...,0
3,أنقذت شجاعة مواطن اليوم فريقًا لشركة متعاقدة م...,0
4,استقبل مدير الدفاع المدني بمنطقة الرياض، اللوا...,0


In [12]:
def suggest(article, n):
    # where n number of suggestions
    pred_label = pipeline.predict([clean(article)])[0]
    recommended = new_data[new_data['labels'] == pred_label][:n]
    return recommended

In [13]:
s = 'محمد بن سلمان'
suggest(s, 4)

,news_text,labels
9,كشف رئيس اللجنة العليا لمشروع قياس وتحقيق رضا ...,8
20,عدَّ وزير الدولة عضو مجلس الوزراء لشؤون مجلس ا...,8
24,استقبل صاحب السمو الملكي الأمير محمد بن سلمان ...,8
36,استقبل وزير الخارجية الأمير فيصل بن فرحان بن ع...,8


In [14]:
s = input()
suggest(s, 4)

محمد بن سلمان


,news_text,labels
9,كشف رئيس اللجنة العليا لمشروع قياس وتحقيق رضا ...,8
20,عدَّ وزير الدولة عضو مجلس الوزراء لشؤون مجلس ا...,8
24,استقبل صاحب السمو الملكي الأمير محمد بن سلمان ...,8
36,استقبل وزير الخارجية الأمير فيصل بن فرحان بن ع...,8


In [15]:
s = 'الترفيه'
suggest(s, 10)

,news_text,labels
23,كرّست القيادة الرشيدة بتأسيسها هيئة تطوير بواب...,7
58,"قدّم صاحب مقطع ""ونتر وندرلاند"" اعتذاره عن الخط...",7
90,أكد الرئيس التنفيذي لهيئة تطوير بوابة الدرعية...,7
114,واصل حساب رئيس الهيئة العامة للترفيه رئيس موسم...,7
128,نظمت الخطوط الجوية العربية السعودية ورشة عمل ...,7
145,حقق موسم الرياض رغبة عشاق المرتفعات من خلال فع...,7
153,بتوجيه أمير المنطقة الشرقية، الأمير سعود بن نا...,7
171,يشهد منتدى الإعلام السعودي الذي يُعقد في ديسمب...,7
197,تشارك وزارة الصحة، بالتعاون مع الهيئة العامة ل...,7
199,أوضح رئيس الهيئة العامة للترفيه تركي آل الشيخ،...,7


In [16]:
s = input()
suggest(s, 6)

الترفيه


,news_text,labels
23,كرّست القيادة الرشيدة بتأسيسها هيئة تطوير بواب...,7
58,"قدّم صاحب مقطع ""ونتر وندرلاند"" اعتذاره عن الخط...",7
90,أكد الرئيس التنفيذي لهيئة تطوير بوابة الدرعية...,7
114,واصل حساب رئيس الهيئة العامة للترفيه رئيس موسم...,7
128,نظمت الخطوط الجوية العربية السعودية ورشة عمل ...,7
145,حقق موسم الرياض رغبة عشاق المرتفعات من خلال فع...,7


In [17]:
s = 'النادي الاهلي'
suggest(s, 7)

,news_text,labels
7,"أعلن المدير الفني للمنتخب السعودي الأول ""رينار...",16
18,بدأ مساء اليوم بيع تذاكر اعتزال ياسر القحطاني ...,16
98,أكد نعيم البكر، رئيس لجنة المسابقات بالاتحاد ا...,16
116,تعادل المنتخب السعودي الأول لكرة القدم سلبياً ...,16
130,أعلن ياسر القحطاني مهاجم فريق الهلال السابق، م...,16
136,تستضيف العاصمة اليابانية طوكيو يوم 23 نوفمبر ...,16
235,أعلن تركي آل الشيخ رئيس الهيئة العامة للترفيه ...,16


In [18]:
s = input()
suggest(s, 12)

نادي الهلال


,news_text,labels
7,"أعلن المدير الفني للمنتخب السعودي الأول ""رينار...",16
18,بدأ مساء اليوم بيع تذاكر اعتزال ياسر القحطاني ...,16
98,أكد نعيم البكر، رئيس لجنة المسابقات بالاتحاد ا...,16
116,تعادل المنتخب السعودي الأول لكرة القدم سلبياً ...,16
130,أعلن ياسر القحطاني مهاجم فريق الهلال السابق، م...,16
136,تستضيف العاصمة اليابانية طوكيو يوم 23 نوفمبر ...,16
235,أعلن تركي آل الشيخ رئيس الهيئة العامة للترفيه ...,16
238,تم تعيين المعلق التونسي عصام الشوالي للتعليق ع...,16
261,زار الأمير عبدالعزيز بن تركي الفيصل، رئيس مجلس...,16
262,يستضيف المنتخب السعودي الأول لكرة القدم منتخب...,16


### Finding Optimal K Clusters Using Elbow Method
I have tried to use Elbow method to find optimal K clusters.

In [20]:
# # selecting K:
# inertias = []
# n_range = range(2, 20)
# for n in n_range:
#     model = KMeans(n_clusters=n)
#     model.fit(X)
#     inertia = model.inertia_
#     inertias.append(inertia)

# inertia
# # Plot the elbow
# plt.plot(range(2, 20), inertias, 'bx-')
# plt.xlabel('k')
# plt.ylabel('Inertias')
# plt.title('The Elbow Method showing the optimal k')
# plt.show()